In [2]:
import json
from pprint import pprint
import re
import numpy as np
from embedding import *

In [3]:
import tensorflow as tf

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [4]:
data = json.load(open('train-v1.1.json'))

In [5]:
def extractor(data):
    contexts = []
    qas = []
    for i in range(len(data["data"])):
        for j in range(len(data["data"][i]["paragraphs"])):
            contexts.append(data["data"][i]["paragraphs"][j]["context"])
            qas.append(data["data"][i]["paragraphs"][j]["qas"])
    return (contexts,qas)

In [6]:
CapPassage = False

from nltk.tokenize import word_tokenize
contexts,qas = extractor(data)

def find_sub_list(sl,l):
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            return ind,ind+sll
    return (-1,-1)

def find_sub_list_2(sl ,l):
    start = 0
    end = len(sl)
    while end < len(l):
        if l[start:end] == sl:
            return (start,right)
        start,end = start+1,end+1
    return (-1,-1)
    

def capPassage(passage,answer,cap_length = 30):
    y = np.zeros(cap_length)
    left,right = find_sub_list(answer,passage)
    if(left==-1):
        return passage[0:cap_length]
    left = left - int((cap_length - len(answer))/2)
    right = right + int((cap_length + len(answer))/2)
    if(left < 0):
        left = 0
    if(right > len(passage)):
        right = len(passage)
    return passage[left:right]
    
def findAnsVec(answer,passage):
    ans = np.zeros((len(passage)))
    start,end = find_sub_list(answer,passage)
    if(start==-1):
        start = passage.index(answer[0])
        end = start + len(answer)
    ans[start:end] = 1
    return ans

In [7]:
X_train_comp_all = []
X_train_comp_ans_all = []
X_train_ans_all = []
Y_train_ques_all = []
invalid = 0
X_train_ans_label_all = []
for i,context in enumerate(contexts):
    passage = word_tokenize(context.lower())
    
    a_lab = np.zeros(len(passage))
    for j,_ in enumerate(qas[i]):
        answer = word_tokenize(qas[i][j]["answers"][0]['text'].lower())
        start,end = find_sub_list(answer,passage)
        if start == -1:
            continue
        a_lab[start:end+1] = 1
            
            
    for j,_ in enumerate(qas[i]):
        try:
            question = word_tokenize(qas[i][j]['question'].lower())
            answer = word_tokenize(qas[i][j]["answers"][0]['text'].lower())
            
            if CapPassage:
                cappedPassage = capPassage(passage,answer)
            else:
                cappedPassage = passage
            
            X_train_comp_ans_all.append(findAnsVec(answer,passage))
            X_train_ans_label_all.append(a_lab)
            X_train_comp_all.append(cappedPassage)
            X_train_ans_all.append(answer)
            Y_train_ques_all.append(question)
        except Exception as e:
            invalid = invalid+1
    

In [8]:
print(invalid)
for i in np.where(X_train_ans_label_all[110] == 1)[0]:
    print(X_train_comp_all[110][i])

1493
rev
.
john
j.
cavanaugh
,
c.s.c
.
served
more
than
half
,
lobund
institute
for
animal
studies
and
medieval
institute
.
hall
of
liberal
arts
(


In [9]:
print(X_train_comp_all[0])
print(X_train_ans_all[0])

['architecturally', ',', 'the', 'school', 'has', 'a', 'catholic', 'character', '.', 'atop', 'the', 'main', 'building', "'s", 'gold', 'dome', 'is', 'a', 'golden', 'statue', 'of', 'the', 'virgin', 'mary', '.', 'immediately', 'in', 'front', 'of', 'the', 'main', 'building', 'and', 'facing', 'it', ',', 'is', 'a', 'copper', 'statue', 'of', 'christ', 'with', 'arms', 'upraised', 'with', 'the', 'legend', '``', 'venite', 'ad', 'me', 'omnes', "''", '.', 'next', 'to', 'the', 'main', 'building', 'is', 'the', 'basilica', 'of', 'the', 'sacred', 'heart', '.', 'immediately', 'behind', 'the', 'basilica', 'is', 'the', 'grotto', ',', 'a', 'marian', 'place', 'of', 'prayer', 'and', 'reflection', '.', 'it', 'is', 'a', 'replica', 'of', 'the', 'grotto', 'at', 'lourdes', ',', 'france', 'where', 'the', 'virgin', 'mary', 'reputedly', 'appeared', 'to', 'saint', 'bernadette', 'soubirous', 'in', '1858.', 'at', 'the', 'end', 'of', 'the', 'main', 'drive', '(', 'and', 'in', 'a', 'direct', 'line', 'that', 'connects', 't

In [9]:
find_sub_list(X_train_ans_all[0] , X_train_comp_all[0])

(102, 105)

In [10]:
print(invalid)
print(X_train_comp_all[101])
print(X_train_ans_all[101])
print(Y_train_ques_all[101])

c = list(zip(X_train_comp_all,X_train_comp_ans_all, X_train_ans_all, X_train_ans_label_all,Y_train_ques_all))
np.random.shuffle(c)
X_train_comp_all_shuffled,X_train_comp_ans_all_shuffled, X_train_ans_shuffled, X_train_ans_label_shuffled,Y_train_ques_all_shuffled = zip(*c)

print(X_train_comp_all_shuffled[101])
print(X_train_comp_ans_all_shuffled[101])
print(X_train_ans_shuffled[101])
print(X_train_ans_label_shuffled[101])
print(Y_train_ques_all_shuffled[101])


1493
['the', 'success', 'of', 'its', 'football', 'team', 'made', 'notre', 'dame', 'a', 'household', 'name', '.', 'the', 'success', 'of', 'note', 'dame', 'reflected', 'rising', 'status', 'of', 'irish', 'americans', 'and', 'catholics', 'in', 'the', '1920s', '.', 'catholics', 'rallied', 'up', 'around', 'the', 'team', 'and', 'listen', 'to', 'the', 'games', 'on', 'the', 'radio', ',', 'especially', 'when', 'it', 'knocked', 'off', 'the', 'schools', 'that', 'symbolized', 'the', 'protestant', 'establishment', 'in', 'america', '—', 'harvard', ',', 'yale', ',', 'princeton', ',', 'and', 'army', '.', 'yet', 'this', 'role', 'as', 'high-profile', 'flagship', 'institution', 'of', 'catholicism', 'made', 'it', 'an', 'easy', 'target', 'of', 'anti-catholicism', '.', 'the', 'most', 'remarkable', 'episode', 'of', 'violence', 'was', 'the', 'clash', 'between', 'notre', 'dame', 'students', 'and', 'the', 'ku', 'klux', 'klan', 'in', '1924.', 'nativism', 'and', 'anti-catholicism', ',', 'especially', 'when', 'dire

In [11]:
examples_to_take_train = 500

X_train_comp = X_train_comp_all_shuffled[0:examples_to_take_train]
X_train_comp_ans = X_train_comp_ans_all_shuffled[0:examples_to_take_train]
X_train_ans = X_train_ans_shuffled[0:examples_to_take_train]
X_train_ans_label = X_train_ans_label_shuffled[0:examples_to_take_train]
Y_train_ques = Y_train_ques_all_shuffled[0:examples_to_take_train]
answer_indices = [np.where(x==1)[0].tolist() for x in X_train_comp_ans]

In [12]:
max_document_len = len(max(X_train_comp,key=len))
max_answer_len = len(max(X_train_ans,key=len))
max_question_len = len(max(Y_train_ques,key=len)) + 1

In [13]:
batch_size = 32
numBatches = examples_to_take_train // batch_size

In [14]:
trainingData = []

for n in range(numBatches):
    batch_dict = {}
    document_tokens = np.zeros((batch_size, max_document_len), dtype=np.int32)
    document_lengths = np.zeros(batch_size, dtype=np.int32)
    answer_labels = np.zeros((batch_size, max_document_len), dtype=np.int32)
    answer_masks = np.zeros((batch_size, max_answer_len, max_document_len), dtype=np.int32)
    answer_lengths = np.zeros(batch_size, dtype=np.int32)
    question_input_tokens = np.zeros((batch_size, max_question_len), dtype=np.int32)
    question_output_tokens = np.zeros((batch_size, max_question_len), dtype=np.int32)
    question_lengths = np.zeros(batch_size, dtype=np.int32)
    question_lengths2 = np.full(batch_size, max_question_len,dtype=np.int32)
    
    for i in range(batch_size):
        index_in_main = (n * batch_size) + i
        answer_labels[i,0:len(X_train_ans_label[index_in_main])] = X_train_ans_label[index_in_main]
        for j, word in enumerate(X_train_comp[index_in_main]):
            document_tokens[i, j] = look_up_word(word)
        document_lengths[i] = len(X_train_comp[index_in_main])
        
        for j, index in enumerate(answer_indices[index_in_main]):
            answer_masks[i, j, index] = 1
        answer_lengths[i] = len(answer_indices[index_in_main])
        
        question_input_words = ([START_WORD] + Y_train_ques[index_in_main])
        question_output_words = (Y_train_ques[index_in_main] + [END_WORD])

        for j, word in enumerate(question_input_words):
                question_input_tokens[i, j] = look_up_word(word)
        for j, word in enumerate(question_output_words):
            question_output_tokens[i, j] = look_up_word(word)
        question_lengths[i] = len(question_input_words)
        
    batch_dict["document_tokens"] = document_tokens
    batch_dict["document_lengths"] = document_lengths
    batch_dict["answer_labels"] = answer_labels
    batch_dict["answer_masks"] = answer_masks
    batch_dict["answer_lengths"] = answer_lengths
    batch_dict["question_input_tokens"] = question_input_tokens
    batch_dict["question_output_tokens"] = question_output_tokens
    batch_dict["question_lengths"] = question_lengths
    batch_dict["question_lengths2"] = question_lengths2

    trainingData.append(batch_dict)

In [15]:
len(trainingData)

15

In [15]:
def create_vocabulary(data):
    flat_list = [item for sublist in data for item in sublist]
    vocabulary = sorted(set(flat_list))
    vocabulary.append("<UNK>")
    vocabulary.append("unk")
    vocabulary.append("eos")
    vocabulary = ["<EOS>"] + vocabulary
    word_to_index = { word:i for i,word in enumerate(vocabulary) }
    index_to_word = { i:word for i,word in enumerate(vocabulary) }
    return (vocabulary,word_to_index,index_to_word)

In [17]:
question_input_tokens[0]

array([    1,    16,    45,  1260,   122,  5091,  1095,  2749,     3,
       15583,   191,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0], dtype=int32)

In [18]:
glove.shape

(400003, 100)

In [16]:
vocabulary_comp,word_to_index_comp,index_to_word_comp = create_vocabulary(X_train_comp + Y_train_ques)
print(len(vocabulary_comp))
print(word_to_index_comp["?"])
print(word_to_index_comp["what"])

12141
819
11800


In [17]:
def create_one_hot_vector(data,vocabulary,word_to_index,index_to_word, maxLen):
    one_hot = np.zeros([maxLen,len(vocabulary)])
    for i,word in enumerate(data):
        if i >= maxLen:
            break
        if(word not in word_to_index):
            word = "<UNK>"
        one_hot[i][word_to_index[word]] = 1
    return one_hot

def create_one_hot_vector_from_indices(data,maxLen,vocabulary):
    one_hot = np.zeros([maxLen,len(vocabulary)])
    for i,indice in enumerate(data):
        if i >= maxLen:
            break
        one_hot[i][int(indice)] = 1
    return one_hot


def create_one_hot_training_Set(data,maxLen,vocabulary):
    one_hot_data = np.zeros([data.shape[0],maxLen,len(vocabulary)])
    for i in range(data.shape[0]):
        one_hot_data[i] = create_one_hot_vector_from_indices(data[i],maxLen,vocabulary)
    return one_hot_data




In [18]:
def sentences_to_indices_glove(X,max_len):
    
    m = len(X)                                 
    
    X_indices = np.full([m,max_len],look_up_word(END_WORD))
    
    for i in range(m):
        j = 0
        for w in X[i]:
            if(j>=max_len):
                break;
            
            X_indices[i, j] = look_up_word(w)
            j = j+1
    return X_indices

In [19]:
document_tokens = sentences_to_indices_glove(X_train_comp, max_document_len)

In [20]:
document_tokens[0]

array([     3,   1912,   6290,     16,   4832,     12,   5877,      4,
            0, 143945,  12476,     26,  14214,    657,     35,   1449,
           27,     17,   2574,     28,      3,  42704,      6,    926,
            7,      3, 154184, 359534,      4,    113,      3,   1625,
         6290,  15926,     16,      3,   3731,      6,     32,    313,
          542,     17,    495,     28,      3,   7761,      6,    282,
        85963,     48, 108327, 343446,      0,      0, 119361,      0,
           26,      3,   1391,    552,      6,   5052,      8,   2748,
           27,      5,   7647,    305,   4832,     12,    242,   2490,
            9,   9528,      4,   5046,    524,      8,    767,     52,
           10,    224,    449,   2897,     15,    320,    112,     71,
         3751,     43,    183,    161,   5227,      5,    376,      4,
         4832,     12,    242,   2490,     17,   7647,      4,    379,
          301,     17,    648,      6,     32,    242,   2490,     22,
      

In [21]:
answer_labels.shape

(32, 373)

In [22]:
tf.reset_default_graph()
embedding = tf.get_variable("embedding", initializer=glove)

EMBEDDING_DIMENS = glove.shape[1]

d_tokens = tf.placeholder(tf.int32, shape=[None, None], name="d_tokens")
d_lengths = tf.placeholder(tf.int32, shape=[None], name="d_lengths")

document_emb = tf.nn.embedding_lookup(embedding, d_tokens)
document_emb = tf.cast(document_emb, dtype=tf.float64)

forward_cell = tf.contrib.rnn.GRUCell(EMBEDDING_DIMENS)
backward_cell = tf.contrib.rnn.GRUCell(EMBEDDING_DIMENS)

# printdocument_emb, document_lengths, dtype=tf.float64,
#     scope="answer_rnn")
answer_outputs, _ = tf.nn.bidirectional_dynamic_rnn(
    forward_cell, backward_cell, document_emb, d_lengths, dtype=tf.float64,
    scope="answer_rnn")

answer_outputs = tf.concat(answer_outputs, 2)

answer_outputs = tf.cast(answer_outputs,tf.float32)

answer_tags = tf.layers.dense(inputs=answer_outputs, units=2)

In [23]:
import tensorflow.contrib.seq2seq as seq2seq

a_labels = tf.placeholder(tf.int32, shape=[None, None], name="a_labels")

answer_mask = tf.sequence_mask(d_lengths, maxlen=max_document_len, dtype=tf.float32)

print(answer_mask)

answer_loss = seq2seq.sequence_loss(
    logits=answer_tags, targets=a_labels, weights=answer_mask, name="answer_loss")

Tensor("SequenceMask/Cast_1:0", shape=(?, 373), dtype=float32)


In [24]:
encoder_input_mask = tf.placeholder(
    tf.float32, shape=[None, None, None], name="encoder_input_mask")
encoder_inputs = tf.matmul(encoder_input_mask, answer_outputs, name="encoder_inputs")
encoder_lengths = tf.placeholder(tf.int32, shape=[None], name="encoder_lengths")

encoder_cell = tf.contrib.rnn.GRUCell(forward_cell.state_size + backward_cell.state_size)

_, encoder_state = tf.nn.dynamic_rnn(
    encoder_cell, encoder_inputs, encoder_lengths, dtype=tf.float32, scope="encoder_rnn")

In [25]:
from tensorflow.python.layers.core import Dense

decoder_inputs = tf.placeholder(tf.int32, shape=[None, None], name="decoder_inputs")
decoder_lengths = tf.placeholder(tf.int32, shape=[None], name="decoder_lengths")
decoder_lengths2 = tf.placeholder(tf.int32, shape=[None], name="decoder_lengths2")

decoder_emb = tf.nn.embedding_lookup(embedding, decoder_inputs)
decoder_emb = tf.cast(decoder_emb,tf.float32)
helper = seq2seq.TrainingHelper(decoder_emb , decoder_lengths2)

projection = Dense(embedding.shape[0], use_bias=False)

decoder_cell = tf.contrib.rnn.GRUCell(encoder_cell.state_size)

decoder = seq2seq.BasicDecoder(decoder_cell, helper, encoder_state, output_layer=projection)

decoder_outputs, _, _ = seq2seq.dynamic_decode(decoder, scope="decoder")
decoder_outputs = decoder_outputs.rnn_output

decoder_labels = tf.placeholder(tf.int64, shape=[None, None], name="decoder_labels")
question_mask = tf.sequence_mask(decoder_lengths, maxlen=max_question_len ,dtype=tf.float32)
question_loss = seq2seq.sequence_loss(
    logits=decoder_outputs, targets=decoder_labels, weights=question_mask,
    name="question_loss")

loss = tf.add(answer_loss, question_loss, name="loss")

In [29]:
optimizer = tf.train.AdamOptimizer().minimize(loss)

saver = tf.train.Saver()
session = tf.Session()
session.run(tf.global_variables_initializer())

EPOCHS = 3
save_every = 1

for epoch in range(1, EPOCHS + 1):
    print("Epoch {0}".format(epoch))
    for i, batch in enumerate(trainingData):
        _, loss_value = session.run([optimizer, loss], {
            d_tokens: batch["document_tokens"],
            d_lengths: batch["document_lengths"],
            a_labels: batch["answer_labels"],
            encoder_input_mask: batch["answer_masks"],
            encoder_lengths: batch["answer_lengths"],
            decoder_inputs: batch["question_input_tokens"],
            decoder_labels: batch["question_output_tokens"],
            decoder_lengths: batch["question_lengths"],
            decoder_lengths2: batch["question_lengths2"],
        })
        print("Loss: {0}".format(loss_value))
        print("Batch: {}/{}".format(i, len(trainingData)))
    
    if epoch % save_every:
        saver.save(session, "model", epoch)

Epoch 1
Loss: 13.518491744995117
Batch: 0/15
Loss: 13.314187049865723
Batch: 1/15
Loss: 13.267959594726562
Batch: 2/15
Loss: 13.21229362487793
Batch: 3/15
Loss: 13.14743423461914
Batch: 4/15
Loss: 13.121570587158203
Batch: 5/15
Loss: 12.850299835205078
Batch: 6/15
Loss: 12.704452514648438
Batch: 7/15
Loss: 12.51314926147461
Batch: 8/15
Loss: 12.187498092651367
Batch: 9/15
Loss: 11.986504554748535
Batch: 10/15
Loss: 11.642542839050293
Batch: 11/15
Loss: 11.3447265625
Batch: 12/15
Loss: 11.076167106628418
Batch: 13/15
Loss: 10.790901184082031
Batch: 14/15
Epoch 2
Loss: 9.618350982666016
Batch: 0/15
Loss: 9.328636169433594
Batch: 1/15
Loss: 8.936561584472656
Batch: 2/15
Loss: 8.665152549743652
Batch: 3/15
Loss: 8.391412734985352
Batch: 4/15
Loss: 8.13070011138916
Batch: 5/15
Loss: 7.841283798217773
Batch: 6/15
Loss: 7.768220901489258
Batch: 7/15
Loss: 7.497920989990234
Batch: 8/15
Loss: 7.175226211547852
Batch: 9/15
Loss: 7.293208599090576
Batch: 10/15
Loss: 7.056112766265869
Batch: 11/15

In [30]:
saver = tf.train.Saver()
saver.save(session, "./model", epoch)

'./model-3'

In [31]:
import numpy as np

# session = tf.Session()

# batch = next(test_data())
# batch = collapse_documents(batch)

answers = session.run(answer_tags, {
    d_tokens: document_tokens,
    d_lengths: document_lengths,
})
answers = np.argmax(answers, 2)

InvalidArgumentError: len(seq_lens) != input.dims(0), (32 vs. 500
	 [[Node: answer_rnn/bw/ReverseSequence = ReverseSequence[T=DT_DOUBLE, Tlen=DT_INT32, batch_dim=0, seq_dim=1, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Cast, _arg_d_lengths_0_0/_365)]]
	 [[Node: answer_rnn/fw/fw/stack/_385 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_90_answer_rnn/fw/fw/stack", tensor_type=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'answer_rnn/bw/ReverseSequence', defined at:
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site

In [32]:
answers.shape

NameError: name 'answers' is not defined

In [43]:
saver = tf.train.Saver()
session = tf.Session()
saver.restore(session, "./model-3")


INFO:tensorflow:Restoring parameters from ./model-3


In [31]:
for k, v in bbatch.items():
    print(k, v.shape)
# print(bbatch)
''' ["document_tokens"],
    d_lengths: bbatch["document_lengths"],
    a_labels: bbatch["answer_labels"],
    encoder_input_mask: bbatch["answer_masks"],
    encoder_lengths: bbatch["answer_lengths"],
'''

document_tokens (32, 373)
document_lengths (32,)
answer_labels (32, 373)
answer_masks (32, 27, 373)
answer_lengths (32,)
question_input_tokens (32, 36)
question_output_tokens (32, 36)
question_lengths (32,)
question_lengths2 (32,)


' ["document_tokens"],\n    d_lengths: bbatch["document_lengths"],\n    a_labels: bbatch["answer_labels"],\n    encoder_input_mask: bbatch["answer_masks"],\n    encoder_lengths: bbatch["answer_lengths"],\n'

In [44]:
import itertools

nb_training_exaples = batch_size = 32
helper = seq2seq.GreedyEmbeddingHelper(embedding, tf.fill([nb_training_exaples], START_TOKEN), END_TOKEN)
decoder = seq2seq.BasicDecoder(decoder_cell, helper, encoder_state, output_layer=projection)
decoder_outputs, _, _ = seq2seq.dynamic_decode(decoder, maximum_iterations=16)
decoder_outputs = decoder_outputs.rnn_output
bbatch = trainingData[2]

questions = session.run(decoder_outputs, {
    d_tokens: bbatch["document_tokens"],
    d_lengths: bbatch["document_lengths"],
    a_labels: bbatch["answer_labels"],
    encoder_input_mask: bbatch["answer_masks"],
    encoder_lengths: bbatch["answer_lengths"],
})



In [53]:
for i in range(nb_training_exaples):
    print(look_up_token(np.argsort(questions[7][1])[-i]))

split-screen
<END>
?
what
the
of
in
is
to
did
was
who
how
a
when
's
which
for
and
are
do
on
many
by
,
does
as
with
that
type
were
where


In [46]:
questions[:,:,UNKNOWN_TOKEN] = 0
qs = np.argmax(questions, 2)
# qs = np.argsort(questions, 2)[-2]

In [49]:
# q = set()
l = []
for i in range(32):
    question = itertools.takewhile(lambda t: t != END_TOKEN, qs[i])
#     l.append([look_up_token(token) for token in question])
#     a =  " ".join(look_up_token(token) for token in question)
    print("Question: " + " ".join(look_up_token(token) for token in question))
#     print("Answer: " + batch["answer_text"][i])
# q

Question: what
Question: what
Question: what
Question: what
Question: what
Question: what
Question: what
Question: what
Question: what
Question: what
Question: what
Question: what
Question: what
Question: what
Question: what
Question: what
Question: what
Question: what
Question: what
Question: what
Question: what
Question: what
Question: what
Question: what
Question: what
Question: what
Question: what
Question: what
Question: what
Question: what
Question: what
Question: what


In [50]:

for i in range()
index_in_main = (30 * batch_size) + i

Y_train_ques[index_in_main]

SyntaxError: invalid syntax (<ipython-input-50-789a31b4bce2>, line 2)